In [1]:
import numpy as np
import pandas as pd
import pulp as lp
import pickle as pkl
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import ALMPlanner as ALM
import ALMChart as ALMc
from ALMChart import standardized_chart

In [2]:
######
EX = 1
######


if EX == 1:
    problem = ALM.ALMplanner(start = "2021", end = "2041", user_risk_profile = 1)
    problem.liabilities.insert("car", "2026", 30000, 30000*0.65)
    problem.liabilities.insert("university", "2029", 50000, 50000*0.95)
    problem.liabilities.insert("hawaii", "2037",30000, 30000*0.85) 
    problem.assets.insert("init","Jan 2021",30000)
    ALM.add_recurrent(problem, start = "Jan 2022", end = "Jan 2027", type = "asset", value = 10000, label = "ass")
elif EX == 2:
    problem = ALM.ALMplanner(start = "Jan 2021", end = "Jan 2061", user_risk_profile = 1)
    ALM.add_recurrent(problem, start = "Jan 2021", end = "Jan 2040", type = "asset", value = 1000, label = "ass")
    ALM.add_recurrent(problem, start = "Jan 2041", end = "Jan 2060", type = "goal", value_tg = 1500, value_lb = 1100, label = "ret")

In [3]:
ALMc.display(problem,bar_width = 6)

In [4]:
problem.check_feasibility()
problem.get_feasibility()

check feasibility ended in 9.53 s with Optimal solution
Unfeasible problem: suggested +2.54%  on assets


In [5]:
ALMc.display(problem,bar_width = 6)

In [6]:
GB_model = ALM.ALMGoalBased(problem)
GB_model.solve(problem)

BaH_model = ALM.ALMBuyAndHold(problem)
BaH_model.solve(problem)

GoalBased model generated in 9.85 s
Solve ended in 1.67 s with Optimal solution
BuyAndHold model generated in 11.18 s
Solve ended in 0.76 s with Optimal solution


# Display solution

In [7]:
sol = GB_model.solution
sol_bah = BaH_model.solution


In [8]:
P = problem.P
A = problem.assets.set
L = problem.liabilities.set
N = problem.N
T = problem.T
Scenario = problem.Scenario

In [9]:
colormap_liab = {}
colormap_ETF = {}
Ltot = list(L)
Ltot.append("extra_wealth")
it = -1
for e in Ltot:
    it = it+1
    colormap_liab[e] = px.colors.qualitative.Plotly[it%10]
it = -1
for e in P:    
    it = it+1
    colormap_ETF[e] = px.colors.qualitative.Plotly[it%10]

In [10]:
def AssetSplitDetailsChart(planner, solution, groupby, colormap):
    P = planner.P
    A = planner.assets.set
    L = planner.liabilities.set
    Assets_split = pd.DataFrame(index = np.arange(len(P)*len(A)*(len(L)+1)), columns = ["Asset", "Goal", "ETF", "Value"])
    it= -1
    for a in A:
        for p in P:
            for l in L:
                it = it+1
                Assets_split["Asset"][it] = a
                Assets_split["Goal"][it] = l
                Assets_split["ETF"][it] = p
                Assets_split["Value"][it] = solution.asset_to_goal[a][l][p]
            it = it+1
            Assets_split["Asset"][it] = a
            Assets_split["Goal"][it] = "extra_wealth"
            Assets_split["ETF"][it] = p
            Assets_split["Value"][it] = solution.asset_to_exwealth[a][p]
    AssetGroupedBy = Assets_split[["Asset", groupby, "Value"]].groupby(by=[groupby, "Asset"]).sum().reset_index()
    AssetPivot = AssetGroupedBy.pivot(index = "Asset", columns=groupby, values = "Value")
    AssetPivot["Period"] = pd.Series(planner.assets.period)
    AssetPivot = AssetPivot.sort_values(by = "Period")
    data = []
    if groupby == "Goal":
        groupby_set = list(L)
        groupby_set.append("extra_wealth")
    elif groupby == "ETF":
        groupby_set = P
    else:
        return
    data = []
    for e in groupby_set:
        data.append(go.Bar(x = AssetPivot.index, y = AssetPivot[e], marker_color = colormap[e], name = e))
    ASDChart = go.Figure(data = data, layout = go.Layout(barmode = "stack"))
    ASDChart = standardized_chart(ASDChart)
    return ASDChart

In [11]:
plt = AssetSplitDetailsChart(problem, sol, "Goal", colormap_liab)
plt.show()
plt = AssetSplitDetailsChart(problem, sol_bah, "Goal", colormap_liab)
plt.show()

In [12]:
plt = AssetSplitDetailsChart(problem, sol, "ETF", colormap_ETF)
plt.show()
plt = AssetSplitDetailsChart(problem, sol_bah, "ETF", colormap_ETF)
plt.show()

In [13]:
n_scen = 1
perc = False
#print([solution[l][n_scen].varValue for l in Lt])
plt = ALMc.AssetAllocationChart(problem,sol,n_scen=n_scen, perc = perc)
plt.show()
plt = ALMc.AssetAllocationChart(problem,sol_bah,n_scen=n_scen, perc = perc)
plt.show()

In [14]:
perc = False
prob, avg = ALMc.GoalRiskDetails(problem, sol, perc)
prob.show()
avg.show()

In [15]:
prob_bah, avg_bah = ALMc.GoalRiskDetails(problem, sol_bah, perc)
prob_bah.show()

avg_bah.show()

In [40]:
planner = problem
solution = sol

L = planner.liabilities.set
N = planner.N
Q_ln = {}
Qex_ln = {}
for l in L:
    Q_ln[l] = np.zeros(shape = (len(N)))
    Qex_ln[l] = np.zeros(shape = (len(N)))
    for n in N:
        Q_ln[l][n] = solution.goal_distr[l][n]
        Qex_ln[l][n] = solution.goal_exwealth[l][n]
Q_ln["ex_wealth"] = np.zeros(shape = (len(N)))
for n in N:
    Q_ln["ex_wealth"][n]=solution.final_exwealth[n]
df_Q_ln = pd.DataFrame(Q_ln)
df_Qex_ln = pd.DataFrame(Qex_ln)
#print(df_Q_ln.head())
#print(df_Qex_ln.head())

conf_tg = pd.DataFrame(planner.liabilities.value_tg, columns = planner.liabilities.set, index = N)
conf_lb = pd.DataFrame(planner.liabilities.value_lb, columns = planner.liabilities.set, index = N)
fail_prob = np.round(np.any((df_Q_ln[planner.liabilities.set] < conf_lb), axis = 1).mean()*100, 1)
success_prob = 100 - fail_prob
print(success_prob, fail_prob)

paid_advance = np.round(np.mean(np.sum(df_Q_ln[L], axis = 1)),0)
extra_wealth = np.round(np.mean(df_Q_ln["ex_wealth"]),0)
print(int(paid_advance), int(extra_wealth))

94.7 5.3
97568 13538


In [ ]:
def show_named_plotly_colours():
    """
    function to display to user the colours to match plotly's named
    css colours.

    Reference:
        #https://community.plotly.com/t/plotly-colours-list/11730/3

    Returns:
        plotly dataframe with cell colour to match named colour name

    """
    s='''
        aliceblue, antiquewhite, aqua, aquamarine, azure,
        beige, bisque, black, blanchedalmond, blue,
        blueviolet, brown, burlywood, cadetblue,
        chartreuse, chocolate, coral, cornflowerblue,
        cornsilk, crimson, cyan, darkblue, darkcyan,
        darkgoldenrod, darkgray, darkgrey, darkgreen,
        darkkhaki, darkmagenta, darkolivegreen, darkorange,
        darkorchid, darkred, darksalmon, darkseagreen,
        darkslateblue, darkslategray, darkslategrey,
        darkturquoise, darkviolet, deeppink, deepskyblue,
        dimgray, dimgrey, dodgerblue, firebrick,
        floralwhite, forestgreen, fuchsia, gainsboro,
        ghostwhite, gold, goldenrod, gray, grey, green,
        greenyellow, honeydew, hotpink, indianred, indigo,
        ivory, khaki, lavender, lavenderblush, lawngreen,
        lemonchiffon, lightblue, lightcoral, lightcyan,
        lightgoldenrodyellow, lightgray, lightgrey,
        lightgreen, lightpink, lightsalmon, lightseagreen,
        lightskyblue, lightslategray, lightslategrey,
        lightsteelblue, lightyellow, lime, limegreen,
        linen, magenta, maroon, mediumaquamarine,
        mediumblue, mediumorchid, mediumpurple,
        mediumseagreen, mediumslateblue, mediumspringgreen,
        mediumturquoise, mediumvioletred, midnightblue,
        mintcream, mistyrose, moccasin, navajowhite, navy,
        oldlace, olive, olivedrab, orange, orangered,
        orchid, palegoldenrod, palegreen, paleturquoise,
        palevioletred, papayawhip, peachpuff, peru, pink,
        plum, powderblue, purple, red, rosybrown,
        royalblue, saddlebrown, salmon, sandybrown,
        seagreen, seashell, sienna, silver, skyblue,
        slateblue, slategray, slategrey, snow, springgreen,
        steelblue, tan, teal, thistle, tomato, turquoise,
        violet, wheat, white, whitesmoke, yellow,
        yellowgreen
        '''
    li=s.split(',')
    li=[l.replace('\n','') for l in li]
    li=[l.replace(' ','') for l in li]

    import pandas as pd
    import plotly.graph_objects as go

    df=pd.DataFrame.from_dict({'colour': li})
    fig = go.Figure(data=[go.Table(
      header=dict(
        values=["Plotly Named CSS colours"],
        line_color='black', fill_color='white',
        align='center', font=dict(color='black', size=14)
      ),
      cells=dict(
        values=[df.colour],
        line_color=[df.colour], fill_color=[df.colour],
        align='center', font=dict(color='black', size=11)
      ))
    ])

    fig.show()

show_named_plotly_colours()